In [32]:
import flymovie as fm
import numpy as np
import scipy.ndimage as ndi
%load_ext autoreload
%autoreload 2

def calc_nM(stack, mask, im_dims, brightness, kernel_1gfp):
    """Calculate concentration in nM of labeled species."""
    pix = stack[mask > 0]
    num_pix = len(pix)
    sum_ = np.sum(pix)
    rel_brightness = brightness / 33.54
    nmolecules = sum_ / np.sum(kernel_1gfp) / rel_brightness

    # multiply each side length, 1e-27 is 1e-9 meters cubed to convert nm to m, 1000 is liters in a cubic meter
    L = np.product(im_dims) * 1e-27 * 1000 * num_pix # in liters
    mols = nmolecules / 6.022e23
    M = mols / L
    nM = M / 1e-9
    return nM

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Add files and params

In [68]:
stack_bg = 2800
kernel_peak = 30_000
kernel_bg = 800
n_gfps = 120
# 34 for egfp, 93 for mneon, 54 for sfgfp
brightness = 34
kernelfile = '/Users/michaelstadler/Bioinformatics/Projects/rpb1/PSFs/psf_20220519_insitu_nm100x50x50_px27x25x25.pkl'
stackfile = '/Volumes/stad3/2022-03-18/bcd-gfp-em2-11.czi'
kernel_dims = (100, 50, 50)
im_dims = (250, 85, 85)

### Calculate kernel for 1 GFP.

In [69]:
kernel = fm.load_pickle(kernelfile)
kernel = ndi.zoom(kernel, np.divide(kernel_dims, im_dims), order=3)
kernel = (kernel - np.min(kernel)) / (np.max(kernel) - np.min(kernel))
kernel_1gfp = kernel * ((kernel_peak - kernel_bg) / np.max(kernel)) / n_gfps

### Smooth and view stack, trimming in Z if necessary/desired

In [22]:
stack = fm.read_czi(stackfile, swapaxes=False)
stack = stack.astype(float) - stack_bg
stack = stack[18:, :, :]
sm = ndi.gaussian_filter(stack, (2,5,5))
fm.viewer(sm, 5)

interactive(children=(Dropdown(description='Color', index=6, options=('Greens', 'gray', 'magma', 'inferno', 'G…

### Set threshold and make mask.

In [24]:
t = 3500
mask = np.where(sm > t, 1, 0)
fm.viewer([np.where(mask, stack, 0), stack], 10)

interactive(children=(Dropdown(description='Color', index=6, options=('Greens', 'gray', 'magma', 'inferno', 'G…

### Calculate concentration

In [25]:
calc_nM(stack, mask, im_dims, brightness, kernel_1gfp)

105.19209561319803

### bicoid (25k - 60)
/Volumes/stad3/2022-03-18/bcd-gfp-em2-11.czi 155 nM (25k - 60)

### rpb1 (40k - 120)
/Volumes/stad3/2022-03-18/rpb1-gfp-em1-21.czi 385 nM (nc14) (252)

/Volumes/stad3/2022-03-18/rpb1-gfp-em2-02.czi 335 nM (early, maybe nc11) (105)

/Volumes/stad3/2022-03-18/rpb1-gfp-em2-06.czi (early)

### zld (25k - 60)
/Volumes/stad3/2022-03-18/zld-mneon-em1-01.czi 83 nM

/Volumes/stad3/2022-03-18/zld-mneon-em1-10.czi 79 nM

/Volumes/stad3/2022-03-18/zld-mneon-em1-14.czi 75 nM

### h2b (4k - 120)
/Volumes/stad3/2022-03-17/h2b-gfp-em1-11.czi 5000 nM

/Volumes/stad3/2022-03-17/h2b-gfp-em2-08.czi 4977 nM

### cp190 (20k - 60)

/Volumes/stad3/2022-03-24/cp190-em3-01.czi 215 nM

/Volumes/stad3/2022-03-24/cp190-em3-08.czi 218 nM

/Volumes/stad3/2022-03-24/cp190-em3-16.czi 212 nM

/Volumes/stad3/2022-03-24/cp190-em3-24.czi 210 nM

### zld cterm (28k - 60)

26983-1-1-em2-01.czi 86 nM

26983-1-1-em2-01.czi 75.8 nM

26983-1-1-em2-16.czi 60 nM

stretched psf:

/Volumes/stad3/2022-03-21/26983-1-1-em2-01.czi 36.2 nM

/Volumes/stad3/2022-03-21/26983-1-1-em2-04.czi 33.91 nM

/Volumes/stad3/2022-03-21/26983-1-1-em2-14.czi 25.65 nM

/Volumes/stad3/2022-03-21/26983-1-1-em2-16.czi 25.38 nM




In [70]:
ones = np.ones((100,100,100))
cv = ndi.convolve(ones, kernel_1gfp, mode='constant', cval=0)

In [71]:
cv[50,50,50]

29115.1484653122

In [65]:
cv[50,50,50]

24129.677837690222

In [52]:
L = np.product(im_dims) * 1e-27 * 1000 # in liters
mols = 1 / 6.022e23
M = mols / L
nM = M / 1e-9
nM


919.3510760429748

In [72]:
2500/29000 * 919


79.22413793103449

Procedure:

- Measure a bunch of nanocages with fast stacks 100 x 50 x 50
- Use same conditions to image in anterior bicoid nuclei -- just need the mean.
- apply above